
# Vanilla Model Replication

When migrating models, there is an intent to migrate the following things from WS to UC:
 - `WS Registered Model` -> `UC Registered Model`
 - `Notebook Experiments` -> `Workspace Experiments`

The Vanilla Experiment is intended to show a migration example where there are no expected manual interventions by the user.

In [0]:
#%pip install --upgrade "mlflow-skinny[databricks]==2.20.0"
%pip install --upgrade "mlflow[databricks]==2.20.0"
%pip install --upgrade "scikit-learn==1.6.1"
dbutils.library.restartPython()


This replication will do the following tasks:
 * Create a WS Experiment
 * Log four models, then register as Workspace Registered Models

In [0]:
import mlflow
from mlflow.tracking import MlflowClient

# Explicitly set the model registry to the WS model registry, ie NOT "datarbricks-uc"
client = MlflowClient(tracking_uri="databricks",
                      registry_uri="databricks")

model_name="vanilla"
experiment_name = f"/Workspace/experiments/uc_ml_migrate/{model_name}"
#experiment_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()

try:
    experiment_id = client.create_experiment(name=experiment_name[10:])
    experiment = client.get_experiment(experiment_id=experiment_id)
    for i in range(3):
            client.set_experiment_tag(experiment_id, f"exp_tag{i}", i)
except:
    experiment = client.search_experiments(filter_string=f'name="{experiment_name[10:]}"')[0]
    experiment_id = experiment.experiment_id


### Model Prep

In model prep, we are going to create all of the artifacts that will go into logging models including the mode important sk_model `lr`, and `input_example` which will be needed when we log our model.

In [0]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn
from mlflow.entities import Dataset

from mlflow.models import ModelInputExample

# Helper function to compute metrics
def compute_metrics(actual, predicted):
    rmse = mean_squared_error(actual, predicted, squared=False)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)
    return rmse, mae, r2


# Load Iris dataset and prepare the DataFrame
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['quality'] = (iris.target == 2).astype(int)  # Create a binary target for simplicity

# Split into training and testing datasets
train_df, test_df = train_test_split(iris_df, test_size=0.2, random_state=42)

# Load the training dataset with MLflow. We will link training metrics to this dataset.
train_dataset: Dataset = mlflow.data.from_pandas(train_df, name="train")
train_x = train_dataset.df.drop(["quality"], axis=1)
train_y = train_dataset.df[["quality"]]

# Fit a model to the training dataset
lr = ElasticNet(alpha=0.5, l1_ratio=0.5, random_state=42)
lr.fit(train_x, train_y)

# Extract features from the test DataFrame
test_x = test_df.drop(["quality"], axis=1)

# Make predictions
input_example: ModelInputExample = lr.predict(test_x)


### Log Experiment Runs

In this section, we'll create all the logged entities that we'll want in scope for migration, including:
 - Model Artifacts
    - Model input_example
 - Model Tags
 - Model Metrics
 - Model Parameters

**NOTE**: Currently, evaluations, and Evaluation Datasets are not in scope due to complexity of changes to the datasets class and the ability to have evaluations runs separate from model logging runs. The legacy mlflow framework doesn't have a reliable way to marry which evaluation run goes with which model and further a consistant way to recover the evaluation dataset used in each model run. This ambiguiity is addressed in MLFlow 3 preview which now has `logged_model` as a first class entity and evaluation runs run against `logged_models` not an arbitrary experiment run.

**NOTE**: As lower priority, will look to migrate the experiment runs and logged models that exist within the same expereiment_run as there is high confidence that the evaluation run is associated with the logged model in the experiment run.

----

The Model Logging process below, will simply keep creating experiement runs until there are four. These runs should be identical and only vary by the experiment run_id and name.

**NOTE**: This does create exceptions in the experiment run that we want. Specifically, that the `model_signature` isn't populated. We will want this missing information in our replication because we intend to automate the creation of signatures so that model can be future compliant and deployable to `ModelServingV2`.

In [0]:
def exp_runs():
    return client.search_runs(experiment_ids=[experiment_id])

while len(exp_runs()) < 4:
    with mlflow.start_run(experiment_id=experiment_id) as run:
        model_info = mlflow.sklearn.log_model(sk_model=lr,
                                              artifact_path=model_name,
                                              input_example=input_example)
        mlflow.set_tags({f"exp_run_tag_{i}": i for i in range(3)})
        mlflow.log_metrics({f"exp_run_metric_{i}": i for i in range(3)})
        mlflow.log_params({f"exp_run_param_{i}": i for i in range(3)})


# Create workspace Registered Model

When using the legacy `databricks` registry, there isn't support to create the registered model artifact at the same time as registering a model version, thus, we will need to create the registered model as a separate task. Further, to make sure that we have the registered model tags migrated, we will also create those tags here.

In [0]:
def rm_tags():
    return client.search_registered_models(experiment_ids=[experiment_id])[0].tags

mlflow.set_registry_uri("databricks")
registered_model_name = model_name
try:
    client.create_registered_model(name=registered_model_name)
    for i in range(3):
        client.set_registered_model_tag(registered_model_name, f"rm_tag_{i}", i)
except:
    registered_model_name = client.search_registered_models(filter_string=f'name="{registered_model_name}"')[0].name


### Register experiment runs as WS registered model verions

We will now register each of the experiment run models as a new registered model version. We'll got further and assign the stages as follows:
 - Version 3 - `production`
 - Version 4 - `staging`
 - Else: `<None>`


REF: https://mlflow.org/docs/latest/model-registry.html?highlight=stage#migrating-from-stages

In [0]:
def rm_versions():
    return client.search_model_versions(filter_string=f"registered_model_name='{registered_model_name}'")

if len(rm_versions())==0:
    registered_model_versions_meta = [{"name": model_name,
                                       "source": f"{r.info.artifact_uri}/{model_name}",
                                       "run_id": r.info.run_id,
                                       "tags": {f"rm_version_tag_{i}": i for i in range(3)}} for i, r in enumerate(exp_runs())]
    for rmv in registered_model_versions_meta:
        client.create_model_version(**rmv)
    for rmv in rm_versions():
        if rmv.version == '3':
            client.transition_model_version_stage(name=registered_model_name, version=3, stage="Production")
        elif rmv.version == '4':
            client.transition_model_version_stage(name=registered_model_name, version=4, stage="Staging")
        elif rmv.__dict__.get('current_stage', 'None') != 'None':
            client.transition_model_version_stage(name=registered_model_name, version=4, stage="None")

In [0]:
model_versions = client.search_model_versions(filter_string=f"name='{model_name}'")
model_versions[0].__dict__

In [0]:
xxx = client.search_runs(experiment_ids=[experiment_id,])

In [0]:
xxx[0]

In [0]:
dir(client)

In [0]:
run = client.get_run('753b835c1c984533a3c361f275f361b2')

In [0]:
run

In [0]:
exp = client.get_experiment("4086430093976181")

In [0]:
exp

In [0]:
dir(client)

In [0]:
?client.search_registered_models

In [0]:
def rm_versions():
    return client.search_model_versions(filter_string=f"registered_model_name='{registered_model_name}'")

In [0]:
rm_versions()

In [0]:
client.search_model_versions(filter_string=f"run_id='fb1dffb632aa4d0180f3e00abd1ea51f'")